In [ ]:
# original data

# "Anpu": 121.5297306, 	25.18258611
# "Taipei" : 121.514853, 25.037658
# "Yangmingshan" : 121.5445472, 25.16207778
# "NTU" : 121.539416, 25.014278
# "PCCU" : 121.53987, 25.13605
# "Science Education Center" : 121.516506, 	25.096356
# "Shezi": 121.469681, 	25.109508
# "Tianmu": 121.537169, 25.117494
# "Neihu" : 121.57545, 25.079422
# "Datunshan" : 121.522408, 25.175675
# "Xinyi": 121.564597, 25.037822
# "Wenshan" : 121.575728, 25.00235
# "Pingdeng" : 121.577086, 25.129142
# "Songshan" : 121.550428, 25.048711
# "Shipai" : 121.513139, 25.115597
# "Freeway No. 3 - CCTV – S016K": 121.6158, 25.03306
# "Freeway No. 3 - CCTV – A005K": 121.5975, 	25.00194
# "Da'an Forest Park": 121.53628, 25.02955
# "Agriculture Guandu Station": 121.492, 	25.11575




In [8]:
import sqlite3
from geopy.distance import geodesic
import pandas as pd

# Weather stations with their coordinates
weather_stations = {
    "Anpu": (121.5297306, 25.18258611),
    "Taipei": (121.514853, 25.037658),
    "Yangmingshan": (121.5445472, 25.16207778),
    "NTU": (121.539416, 25.014278),
    "PCCU": (121.53987, 25.13605),
    "Science Education Center": (121.516506, 25.096356),
    "Shezi": (121.469681, 25.109508),
    "Tianmu": (121.537169, 25.117494),
    "Neihu": (121.57545, 25.079422),
    "Datunshan": (121.522408, 25.175675),
    "Xinyi": (121.564597, 25.037822),
    "Wenshan": (121.575728, 25.00235),
    "Pingdeng": (121.577086, 25.129142),
    "Songshan": (121.550428, 25.048711),
    "Shipai": (121.513139, 25.115597),
    "Freeway No. 3 - CCTV – S016K": (121.6158, 25.03306),
    "Freeway No. 3 - CCTV – A005K": (121.5975, 25.00194),
    "Da'an Forest Park": (121.53628, 25.02955),
    "Agriculture Guandu Station": (121.492, 25.11575),
}

# Connect to the database
db_path = '../youbike_data.db'
conn = sqlite3.connect(db_path)

# Retrieve YouBike stations
query = "SELECT sno, sna, latitude, longitude FROM youbike_stations"
df_youbike = pd.read_sql_query(query, conn)

# Find the closest weather station for each YouBike station
closest_stations = []
for _, row in df_youbike.iterrows():
    youbike_coords = (row["longitude"], row["latitude"])  # Latitude first, then longitude
    min_distance = float("inf")
    closest_station = None

    for station_name, weather_coords in weather_stations.items():
        distance = geodesic(youbike_coords[::-1], weather_coords[::-1]).kilometers
        if distance < min_distance:
            min_distance = distance
            closest_station = station_name

    closest_stations.append((row["sno"], row["sna"], closest_station, min_distance))

# Create a DataFrame for the results
df_closest = pd.DataFrame(
    closest_stations,
    columns=["YouBike Station ID", "YouBike Station Name", "Closest Weather Station", "Distance (km)"]
)

# Save the results to the database
df_closest.to_sql("station_weather_mapping", conn, if_exists="replace", index=False)

# Close the connection
conn.close()

print("Closest weather stations mapped and saved to the database.")


Closest weather stations mapped and saved to the database.


In [3]:
weather_coords

(121.5297306, 25.18258611)